In [23]:
import scipy
from scipy.linalg import expm
import scipy.sparse as sparse
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [27]:
L = 5 #system size
M = 0.1 #fermion mass
N=1 #number of layers
l=1 #spin length
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [28]:
sigma = np.zeros((3,3))
sigma[0][1] = 0.5
sigma[1][0] = 0.5

tensprod = sigma
for i in range(L-1):
    tensprod = np.kron(np.identity(3),tensprod)
print(tensprod)

[[0.  0.5 0.  ... 0.  0.  0. ]
 [0.5 0.  0.  ... 0.  0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]
 ...
 [0.  0.  0.  ... 0.  0.5 0. ]
 [0.  0.  0.  ... 0.5 0.  0. ]
 [0.  0.  0.  ... 0.  0.  0. ]]


In [1]:
def rot_sigma(k,i,j,phi,theta):
    sigma = np.zeros((3,3),dtype = complex)
    sigma[2-i][2-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2-j][2-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csr_matrix(sigma)
    rot_matrix = expm(-1j*theta*sigma/2)
    if k==0:
        tensprod = rot_matrix
        for i in range(L-1):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
    else:
        tensprod = np.identity(3)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+1,L):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
    return tensprod

def rot_ms(k,i,j,phi,theta):
    sigma = np.zeros((3,3),dtype = complex)
    sigma[2-i][2-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2-i][2-j] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csr_matrix(sigma)
    power_matrix = scipy.sparse.kron(sigma,np.identity(3))+scipy.sparse.kron(np.identity(3),sigma)
    rot_matrix = expm(-1j*power_matrix@power_matrix*theta/4)
    if k == 0:
        tensprod = rot_matrix
        for i in range(L-2):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
    else:
        tensprod = np.identity(3)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+2,L):
            tensprod = scipy.sparse.kron(tensprod,np.identity(3))
    return tensprod



In [9]:
def clause(a,b,k,m,n):
    zero = 0
    m = int(m)
    n = int(n)
    for l in range(L):
        if l == k:
            if (int(a[l])!=m) | (int(b[l])!=n):
                zero +=1
        else:
            if a[l]!=b[l]:
                zero +=1
    return zero

In [ ]:
sigma_01_temp = np.zeros((basis.Ns,basis.Ns),dtype = complex)

for i in range(basis.Ns):
    string_i = basis.int_to_state(basis.Ns-i-1,bracket_notation=False)
    for j in range(basis.Ns):
        string_j = basis.int_to_state(basis.Ns-j-1,bracket_notation=False)

        if (clause(string_i,string_j,0,0,1)==0):
            sigma_01_temp[i][j] = 0.5
        if (clause(string_i,string_j,0,1,0)==0):
            sigma_01_temp[i][j] = 0.5